# LLM Pretraining and SFT (Single Notebook)

This notebook implements all steps required by the PRD in a single place:

- Pretrain on a Russian literature corpus (local `data/corpus/`) to learn language structure.
- Train a custom BPE tokenizer (~3k vocab) with `<unk>`, `<pad>`, `<bos>`, `<eos>`.
- Build tokenized datasets and train a ~150M decoder-only model (context 512) with HF Trainer.
- After each epoch, run deterministic generations on the 10 assignment prompts and display them.
- SFT Qwen2.5-0.5B using local `data/alpaca-cleaned-ru/` in conversational format (system/user/assistant).
- Evaluate on 4 assignment questions and display outputs.

Constraints:
- One notebook only (no external scripts/configs).
- Use `uv` for environment and `ruff` for linting.
- Keep logs/docs in English; dataset strings and model generations remain Russian where appropriate.

Run environment:
- Start via: `uv run jupyter lab` (or `uv run jupyter notebook`).
- Python packages and versions come from `pyproject.toml`/`uv.lock`.



In [ ]:
from __future__ import annotations

import random
import numpy as np
import torch

from transformers import (
    set_seed,
)



In [ ]:
import os


def seed_everything(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    set_seed(seed)


SEED = int(os.environ.get("SEED", "42"))
seed_everything(SEED)

if torch.cuda.is_available():
    device = torch.device("cuda")
    device_name = torch.cuda.get_device_name(0)
    dtype = torch.float16
else:
    device = torch.device("cpu")
    device_name = "CPU"
    dtype = torch.float32

print({
    "seed": SEED,
    "device": str(device),
    "device_name": device_name,
    "cuda": torch.cuda.is_available(),
    "dtype": str(dtype),
})



In [ ]:
from pathlib import Path
import os
import json

CORPUS_DIR = Path("data/corpus")
ALPACA_DIR = Path("data/alpaca-cleaned-ru")

assert CORPUS_DIR.exists() and CORPUS_DIR.is_dir(), f"Missing directory: {CORPUS_DIR}"
assert ALPACA_DIR.exists() and ALPACA_DIR.is_dir(), f"Missing directory: {ALPACA_DIR}"

corpus_txt_files = sorted(p for p in CORPUS_DIR.glob("**/*.txt"))
parquet_files = sorted(p for p in ALPACA_DIR.glob("*.parquet"))

stats = {
    "corpus_num_files": len(corpus_txt_files),
    "corpus_sample": [str(p) for p in corpus_txt_files[:3]],
    "alpaca_parquet_files": [str(p) for p in parquet_files],
}

print(json.dumps(stats, ensure_ascii=False, indent=2))

